### 0. 데이터 적재 및 변수 타입 파악

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.cluster import DBSCAN

In [2]:
# 모든 열 출력
pd.set_option('display.max_columns',None)

In [3]:
df = pd.read_csv('WA_Fn-UseC_-HR-Employee-Attrition.csv')
df.head()

FileNotFoundError: [Errno 2] No such file or directory: 'WA_Fn-UseC_-HR-Employee-Attrition.csv'

In [ ]:
df.shape

- <I>수치형 변수처럼 보이지만 범주형 변수인 경우가 있다<I>
    - 예를 들어, Education은 1~5이지만 이는 교육 수준을 말하며 학사, 석사, 박사처럼 범주형에 해당된다.
    - 이에 따라, 수치형 변수들에서도 범주형 변수가 있는지 확인하고 작업해야한다!

In [ ]:
for i in df.columns:
    print('<',i,'>')
    print(df[i].unique()[:10])
    print('\n')

In [ ]:
# Kaggle Dataset 소개에 따라 수치형 변수의 범주화 진행

df["Education"] = df["Education"].replace({1:"Below College",2:"College",3:"Bachelor",4:"Master",5:"Doctor"})
df["EnvironmentSatisfaction"] = df["EnvironmentSatisfaction"].replace({1:"Low",2:"Medium",3:"High",4:"Very High"})
df["JobInvolvement"] = df["JobInvolvement"].replace({1:"Low",2:"Medium",3:"High",4:"Very High"})
df["JobLevel"] = df["JobLevel"].replace({1:"Entry Level",2:"Junior Level",3:"Mid Level",4:"Senior Level",
                                         5:"Executive Level"})
df["JobSatisfaction"] = df["JobSatisfaction"].replace({1:"Low",2:"Medium",3:"High",4:"Very High"})
df["PerformanceRating"] = df["PerformanceRating"].replace({1:"Low",2:"Good",3:"Excellent",4:"Outstanding"})
df["RelationshipSatisfaction"] = df["RelationshipSatisfaction"].replace({1:"Low",2:"Medium",3:"High",4:"Very High"})
df["WorkLifeBalance"] = df["WorkLifeBalance"].replace({1:"Bad",2:"Good",3:"Better",4:"Best"})

In [ ]:
df.head()

---

### 1. 결측치, 이상치 검토

#### 1-1. 결측치

In [ ]:
# 결측치
missing_values = df.isnull().sum()
missing_values

In [ ]:
for col in df.columns:
    print('<',col,'>')
    print(df[col].value_counts()[:10])
    print('\n')

#### 1-2. 이상치

In [ ]:
# 수치형 변수 컬럼
numeric_cols = df.select_dtypes(include=['int64', 'float64']).columns

constant_cols = [col for col in numeric_cols if df[col].nunique() == 1]
variable_numeric_cols = [col for col in numeric_cols if col not in constant_cols]

# 기초 통계량
df[variable_numeric_cols].describe().T

##### 1-2-1. IQR 방식

- IQR (Interquartile Range)이란?
    - IQR 방식은 데이터의 하위 25%와 상위 25% 사이의 범위를 이용하여 이상치를 감지하고 처리하는 방법입니다.<br>IQR은 다음과 같은 수식으로 계산됩니다:

$$IQR = Q3 - Q1$$
<br><br>

<div style="text-align: center;">
    <img src="../img/iqr.png" width="500"/><br>
    (이미지 출처: <a href="https://en.wikipedia.org/wiki/Interquartile_range">https://en.wikipedia.org/wiki/Interquartile_range</a>)
</div>

여기서:
- Q1은 데이터의 1사분위수 (하위 25%)를 나타냅니다.
- Q3은 데이터의 3사분위수 (상위 25%)를 나타냅니다.

IQR 방식을 이용하면, \(Q1 - 1.5 \times IQR\) 보다 작거나 \(Q3 + 1.5 \times IQR\) 보다 큰 데이터 포인트는 이상치로 간주됩니다. 

IQR 방식은 데이터의 분포가 대칭적이지 않을 때 특히 유용합니다. 그 이유는 IQR 방식이 데이터의 중앙 부분에 초점을 맞추기 때문입니다. 이 방식을 사용하면, 이상치의 영향을 줄이고 데이터의 중앙 부분을 더 잘 대표하는 모델을 만들 수 있습니다.


In [ ]:
# Create a copy of the dataframe to apply outlier removal
df_iqr = df.copy()

# IQR 방식
def handle_outliers_iqr(col, dataframe):
    Q1 = dataframe[col].quantile(0.25)
    Q3 = dataframe[col].quantile(0.75)
    IQR = Q3 - Q1

    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    outliers = dataframe[(dataframe[col] < lower_bound) | (dataframe[col] > upper_bound)]
    outlier_count = outliers.shape[0]

    # Replace outliers with the lower and upper bounds
    dataframe.loc[(dataframe[col] < lower_bound), col] = lower_bound
    dataframe.loc[(dataframe[col] > upper_bound), col] = upper_bound

    return outlier_count

# 시각화
fig, axes = plt.subplots(len(variable_numeric_cols), 2, figsize=(14, len(variable_numeric_cols)*4))

for idx, col in enumerate(variable_numeric_cols):
    # 본 데이터
    sns.boxplot(x=df[col], ax=axes[idx, 0], color='blue')
    axes[idx, 0].set_title(f"Original {col} (Outliers: {handle_outliers_iqr(col, df_iqr)})", 
                            fontsize=14, pad=20)

    # 이상치 처리 후 데이터
    sns.boxplot(x=df_iqr[col], ax=axes[idx, 1], color='green')
    axes[idx, 1].set_title(f"After IQR Outlier Handling {col}", 
                            fontsize=14, pad=20)

plt.tight_layout()
plt.show()

In [ ]:
# IQR 데이터 DF화
for col in variable_numeric_cols:
    handle_outliers_iqr(col, df_iqr)

outlier_indices = df_iqr[df_iqr.isnull().any(axis=1)].index

# 종속 변수가'Yes'인 경우
attrition_yes_outliers = df.loc[outlier_indices, 'Attrition'] == 'Yes'
attrition_yes_outliers_proportion = attrition_yes_outliers.mean()

attrition_yes_outliers_proportion

- 종속 변수가'Yes'인 경우가 없어, IQR 방식으로 추출된 이상치를 제거해도 괜찮을 것 같다<br>하지만 추가적으로 다른 방법도 탐색

##### 1-2-2.  표준점수로 변환 후 -3이하 및 +3 제거

- Z-점수 표준화란?

    - Z-점수는 데이터 포인트가 평균으로부터 표준편차의 몇 배만큼 떨어져 있는지를 나타내는 표준화 방법입니다. Z-점수는 다음과 같은 수식으로 계산됩니다:

$$Z = \frac{(X - μ)}{σ}$$
<br><br>

<div style="text-align: center;">
    <img src="../img/z_score.png" width="500"/><br>
    (이미지 출처: <a href="https://vitalflux.com/z-score-z-statistics-concepts-formula-examples/">https://vitalflux.com/z-score-z-statistics-concepts-formula-examples/</a>)
</div>

여기서:
- X는 각 데이터 포인트를 나타냅니다.
- μ는 데이터의 평균을 나타냅니다.
- σ는 데이터의 표준편차를 나타냅니다.

Z-점수는 데이터를 표준화하여 각 데이터 포인트가 원래 분포의 평균으로부터 얼마나 떨어져 있는지를 측정합니다. 

데이터 포인트의 Z-점수가 3 또는 -3을 초과하는 경우, 이 데이터 포인트는 이상치로 간주됩니다. 이 값은 일반적으로 사용되는 임계값이지만, 필요에 따라 다른 값으로 설정할 수 있습니다.

In [ ]:
from scipy.stats import zscore

# Create a copy of the dataframe to apply outlier removal
df_zscore = df.copy()

# z-score +3, -3 함수
def handle_outliers_zscore(col, dataframe):
    z_scores = zscore(dataframe[col])
    abs_z_scores = np.abs(z_scores)

    outliers = dataframe[abs_z_scores > 3]
    outlier_count = outliers.shape[0]

    # Replace outliers with NaN
    dataframe.loc[(abs_z_scores > 3), col] = np.nan

    return outlier_count

# 시각화
fig, axes = plt.subplots(len(variable_numeric_cols), 2, figsize=(14, len(variable_numeric_cols)*4))

for idx, col in enumerate(variable_numeric_cols):
    # 본 데이터
    sns.distplot(df[col], ax=axes[idx, 0], color='blue', hist=False, kde=True)
    axes[idx, 0].set_title(f"Original {col} (Outliers: {handle_outliers_zscore(col, df_zscore)})", 
                            fontsize=14, pad=20)

    # 이상치 처리 후 데이터
    sns.distplot(df_zscore[col], ax=axes[idx, 1], color='green', hist=False, kde=True)
    axes[idx, 1].set_title(f"After Z-Score Outlier Handling {col}", 
                            fontsize=14, pad=20)

plt.tight_layout()
plt.show()

- 유의미한 변화가 보이지 않아, 다른 방식으로 추가 진행

##### 1-2-3.  표준화 후 PCA와 DBSCAN 알고리즘으로 제거

In [ ]:
# 수치형 컬럼만 선택
numeric_columns = df.select_dtypes(include=['int64', 'float64'])

# 표준화
scaler = StandardScaler()
numeric_scaled = scaler.fit_transform(numeric_columns)

# 수치형 컬럼만 데이터 프레임화
numeric_scaled = pd.DataFrame(numeric_scaled, columns=numeric_columns.columns)

numeric_scaled.head()

- DBSCAN이란?
    - 데이터의 밀도에 기반한 클러스터링 알고리즘입니다.<br>데이터 포인트의 밀도를 계산하여,<br>데이터 포인트의 밀도가 일정 수준 이상이면 그 데이터 포인트를 클러스터의 일부로 간주합니다.

- DBSCAN은 다음과 같은 주요 특징

    - 클러스터의 수를 미리 정의할 필요가 없습니다: DBSCAN은 데이터의 분포에 따라 클러스터의 수를 자동으로 결정합니다.
    - 임의의 형태의 클러스터를 찾을 수 있습니다: DBSCAN은 k-means와 같은 알고리즘과 달리 원형의 클러스터만 찾지 않고, 임의의 형태의 클러스터를 찾을 수 있습니다.
    - 이상치를 처리할 수 있습니다: DBSCAN은 밀도가 낮은 영역의 데이터 포인트를 이상치로 간주하여, 이상치 탐지에도 사용할 수 있습니다.

- DBSCAN은 다음과 같은 두 개의 주요 매개변수

    - `eps`: 이 매개변수는 클러스터의 최대 반경을 정의합니다. `eps` 거리 내에 있는 데이터 포인트의 수가 `min_samples` 이상이면, 그 데이터 포인트를 클러스터의 일부로 간주합니다.
    - `min_samples`: 클러스터를 형성하는 데 필요한 최소 데이터 포인트의 수를 정의합니다.

이러한 특징 때문에 DBSCAN은 공간적 클러스터링과 이상치 탐지에 효과적인 알고리즘이라고 할 수 있습니다.<br><br>

<div style="text-align: center;">
    <img src="../img/dbscan.png" width="500"/><br>
    (이미지 출처: <a href="https://yganalyst.github.io/ml/ML_clustering/">https://yganalyst.github.io/ml/ML_clustering/</a>)
</div>

In [ ]:
# DBSCAN
dbscan = DBSCAN(eps=3.0, min_samples=5)
clusters = dbscan.fit_predict(numeric_scaled)

# 이상치 수
outliers = (clusters == -1).sum() # (clusters == -1) -1일 경우, 클러스터에 속하지 않음

outliers

- PCA (Principal Component Analysis)란?
    - 즉 주성분 분석은 고차원 데이터를 저차원 데이터로 변환하는 데 사용되는 통계적 방법입니다.<br>PCA는 데이터의 분산이 최대가 되는 방향으로 축을 회전시키며, 이렇게 생성된 새로운 축을 주성분이라고 합니다.<br>이 주성분들은 원래의 특성들의 선형 조합으로 이루어져 있습니다.

- PCA 주요 특징

    - 데이터의 차원 축소: PCA는 고차원의 데이터를 저차원의 데이터로 축소시키는 데 주로 사용됩니다. 이는 데이터를 시각화하거나, 머신러닝 모델의 성능을 향상시키는 데 도움이 될 수 있습니다.
    - 정보 손실 최소화: PCA는 원본 데이터의 분산을 최대한 보존하려고 합니다. 이는 원본 데이터의 정보를 최대한 보존하면서 차원을 축소하는 데 도움이 됩니다.
    - 상관관계 감소: PCA는 변환된 데이터의 특성들이 서로 직교하도록 만듭니다. 이는 변환된 특성들 사이의 상관관계를 제거합니다.

PCA는 이러한 특징 때문에 데이터 전처리, 시각화, 특성 추출 등 다양한 분야에서 널리 사용됩니다.

- PCA의 축(차원)
    - 원본 데이터의 분산이 가장 큰 방향을 나타내는 벡터입니다.<br>첫 번째 주성분은 데이터의 분산이 가장 큰 방향을 찾아내고, 두 번째 주성분은 그 다음으로 분산이 큰 방향을 찾아냅니다.<br>이 과정은 원하는 차원의 수만큼 계속되며, 각 주성분은 이전의 주성분들과 직교(orthogonal)하게 설정됩니다.<br><br>

<div style="text-align: center;">
    <img src="../img/pca.png" width="500"/><br>
    (이미지 출처: <a href="https://m.blog.naver.com/sanghan1990/221156213790">https://m.blog.naver.com/sanghan1990/221156213790</a>)
</div>

In [ ]:
# 2차원으로 데이터 축소
pca = PCA(n_components=2)
reduced_data = pca.fit_transform(numeric_scaled)

# Plot the reduced data and highlight the outliers
plt.figure(figsize=(5, 5))

# 이상치인 경우
plt.scatter(reduced_data[clusters != -1, 0], reduced_data[clusters != -1, 1], 
            c=clusters[clusters != -1], cmap='Paired', label='Outlier X')

# 이상치 아닌 경우
plt.scatter(reduced_data[clusters == -1, 0], reduced_data[clusters == -1, 1], 
            color='red', label='Outlier O')

plt.legend()
plt.title('Outlier detection using DBSCAN')
plt.show()

- 시각화된 결과물을 봤을 때<br>대체로 좌하단 방향으로 몰려있으며, 우상단에 가까운 데이터는 이상치로 파악됨을 알 수 있음

<div style="text-align: center;">
    <img src="../img/outlier_detection_dbscan.png" width="400"/>
</div>

In [ ]:
# DBSCAN 알고리즘을 이용하여 파악된 이상치 제거하기

# 이상치에 대한 boolean mask 생성
outlier_mask = (clusters == -1)

# 이상치 데이터 추출
data_without_outliers = numeric_scaled[~outlier_mask]

data_without_outliers.head()

In [ ]:
print('제거된 데이터수: ',len(df)-len(data_without_outliers))

##### 1-2-4.  KMeans 클러스터링 알고리즘으로 제거

In [ ]:
# DBSCAN 진행전 스케일링한 수치형 변수 데이터 프레임 사용
# numeric_scaled


---

### 2. 유의미한 시각화 5개 이상

#### 2-1. 이탈 여부에 따른 직무 만족도 비교

#### 2-2. 이탈 여부에 따른 급여 분포 비교

#### 2-3. 직급에 따른 이탈 비율

#### 2-4. 직무 스트레스 수준과 이탈 간의 상관 관계

#### 2-5. 

---

### 3. 수치형 변수 간 상관관계 파악

In [ ]:
# Calculate the correlation matrix
df_corr = df.drop(columns=['EmployeeCount','StandardHours'])
corr_matrix = df_corr.corr()

# Create a heatmap
plt.figure(figsize=(20,20))
sns.heatmap(corr_matrix, annot=True, fmt=".2f", cmap='coolwarm', linewidths=.5)
plt.title("Correlation Matrix")
plt.show()

---

### 4. 파생변수 생성

- 가상 데이터이지만 한국이라는 가정하에 국내 퇴사 원인 설문조사를 근거로 파생변수 새롭게 생성
<div style="text-align: center;">
    <img src="../img/Attrition_caustion.png" width="500"/><br>
    (이미지 출처: <a href="https://m.blog.naver.com/sanghan1990/221156213790">https://m.blog.naver.com/sanghan1990/221156213790</a>)
</div>


#### 스트레스 지수
- 직무 만족도, 근무 조건 등을 고려하여 계산